In [3]:
!pip install -q label-studio-converter


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [47]:
from pathlib import Path
from datetime import datetime
from pprint import pprint as pp
import os
import json
import copy
import logging

from label_studio_converter import converter


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [48]:
# Labelstudio converter forces to download the images so we can only use this dirty way to avoid it
def cus_download(
    url,
    output_dir,
    filename=None,
    project_dir=None,
    return_relative_path=False,
    upload_dir=None,
    download_resources=True,
):
    return url

converter.download = cus_download

In [49]:
BASE_PATH = "../../tmp/convert_from_ls"
CURRENT_TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")


# Source exported json
ls_src_dir = os.path.join(BASE_PATH, "export_sync_ls")
# final conversionoutput
out_file = os.path.join(BASE_PATH, "output", CURRENT_TIMESTAMP)
os.makedirs(out_file, exist_ok=True)

# For conversion of json from export sync API if set to true
IS_EXPORT_SYNC = True
convert_des = os.path.join(BASE_PATH, 'generated_from_export_sync', CURRENT_TIMESTAMP)
os.makedirs(convert_des, exist_ok=True)


# Annotation Project setting XML
xml_path = os.path.join(BASE_PATH, 'task_config/ocr_invoice.xml')
c = converter.Converter(xml_path, None)


In [50]:
# 1st step conversion of json from export sync API
# Because the JSON from export sync API is not valid for the converter.
# We need to convert it first.
if IS_EXPORT_SYNC:
    json_list = []
    # Load all json(with or without .json suffix) from the source directory and convert restructure it
    # Then append to the json list
    for file in os.listdir(ls_src_dir):
        with open(os.path.join(ls_src_dir, file), 'r') as f:
            j_obj = json.load(f)
            if j_obj:
                # The task object should be put in the root of the json
                new_j_obj = copy.deepcopy(j_obj.get("task", {}))
                del j_obj["task"]
                # The remained fields should be put inside "annotations" list
                new_j_obj["annotations"] = [j_obj]
            json_list.append(new_j_obj)
        print("Finished processing : ", file)
    print("Total processed json files : ", len(json_list))
    # pp(json_list)
    # Store the converted correct json to the middle directory
    with open(os.path.join(convert_des, "corrected_export.json"), 'w') as outfile:
        json.dump(json_list, outfile, ensure_ascii=False, indent=4)
    # Next step will use this path
    ls_convert_src_dir = convert_des
else:
    # If export sync conversion is not needed, just use the source directory
    ls_convert_src_dir = ls_src_dir


Finished processing :  2150
Finished processing :  2152
Finished processing :  2147
Total processed json files :  3


In [51]:
# Convert from json to voc
print("Converting JSON from : ", ls_convert_src_dir)
c.convert_to_voc(ls_convert_src_dir, os.path.join(out_file, "voc"))
print("Converted files are stored in : ", out_file)

Converting JSON from :  ../../tmp/convert_from_ls/generated_from_export_sync/20230906194549
Converted files are stored in :  ../../tmp/convert_from_ls/output/20230906194549
